In [1]:
# Pronóstico usando un modelo mlp con differenciacion
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Carga de datos
import nbimporter  #  type: ignore
import functions  #  type: ignore
df_orig = functions.load_data()
df_orig.head()

In [ ]:
# Remove trend and cycle to make the series stationary
df_orig = functions.remove_trend_and_cycle(df_orig, yt_true_name="yt_true")
df_orig.head(20)

In [ ]:
# Construcción de la matriz de regresores
p_max = 13
df_orig = functions.make_lagged_ts(
    df=df_orig,
    p_max=p_max,
    y_column="yt_true_d1d12",
    fmt="lagged_{}m",
)
df_orig.head(20)

In [ ]:
# Remoción de los valores faltantes
df_dropna = df_orig.dropna()
df_dropna.head()

In [ ]:
# División de los datos en entrenamiento y prueba
(
    X_complete,
    y_complete,
    X_train,
    y_train,
    X_test,
    y_test,
) = functions.train_test_split(
    df=df_dropna,
    x_columns=[f"lagged_{i}m" for i in range(1, 14)],
    y_column="yt_true_d1d12",
)

display(X_complete.head())
display(y_complete.head())

In [7]:
# Pronóstico usando una red neuronal tipo MLP

from sklearn.pipeline import Pipeline  #  type: ignore
from sklearn.preprocessing import MinMaxScaler  #  type: ignore
from sklearn.compose import TransformedTargetRegressor  #  type: ignore
from sklearn.neural_network import MLPRegressor  #  type: ignore

In [8]:
# Crea un pipeline para automatizar la creacion de un modelo
def make_pipeline_from_model(model):
    """Create a pipeline."""
    return Pipeline(
        [
            (
                "scaler",
                MinMaxScaler(),
            ),
            (
                "regressor",
                TransformedTargetRegressor(
                    regressor=model,
                    transformer=MinMaxScaler(),
                ),
            ),
        ]
    )  

In [ ]:
# Entrenamiento y pronostico
hidden = 4
pipeline = make_pipeline_from_model(
    model=MLPRegressor(
        hidden_layer_sizes=(hidden,),
        activation="logistic",
        learning_rate="adaptive",
        momentum=0.01,
        learning_rate_init=0.2,
        max_iter=10000,
        random_state=12345,
    )
)
pipeline.fit(X_train, y_train)
df_dropna[f"yt_d1d12_mlp"] = pipeline.predict(X_complete)
df_orig.loc[df_dropna.index, "yt_d1d12_mlp"] = df_dropna[f"yt_d1d12_mlp"]
df_orig[
    [
        "yt_true",
        "yt_true_d1d12",
        f"yt_d1d12_mlp",
    ]
].head(40)

In [ ]:
# Integración simple y estacional
import numpy as np  #  type: ignore
df_orig = df_orig.assign(yt_pred_d1d12_mlp=np.nan)
df_orig.loc[df_dropna.index, "yt_pred_d1d12_mlp"] = df_dropna["yt_d1d12_mlp"]
df_orig["yt_pred_d1d12_mlp"] += df_orig["yt_true"].shift(1)
df_orig["yt_pred_d1d12_mlp"] += df_orig["yt_true"].shift(12)
df_orig["yt_pred_d1d12_mlp"] -= df_orig["yt_true"].shift(13)
df_orig[["yt_true", "yt_pred_d1d12_mlp"]].head(40)

In [ ]:
# Gráfico de los pronósticos
functions.plot_time_series(df=df_orig, yt_col="yt_true")

In [ ]:
# Almacenamiento de los resultados

functions.save_forecasts(df_orig)
metrics = functions.compute_evaluation_metrics(df_orig.dropna())
functions.save_metrics(metrics)
metrics